In [130]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from sklearn.calibration import calibration_curve
import shap
import matplotlib.pyplot as plt
import joblib
import warnings
import openpyxl
warnings.filterwarnings('ignore')

from lightgbm import LGBMClassifier

from sklearn.metrics import (
    roc_auc_score, roc_curve,
    f1_score, precision_score, recall_score,
    accuracy_score, matthews_corrcoef
)
from sklearn.model_selection import TimeSeriesSplit
from mlflow.models.signature import infer_signature

import lightgbm as lgb
import optuna
import mlflow
import mlflow.lightgbm

In [136]:
df = pd.read_csv('base_modelo.csv', parse_dates=['safra'], dayfirst=True)


### Substituir nulos nas variáveis
### Identificar colunas que contêm valores negativos
### Definir as outras colunas sem valores negativos
### Para as demais, penalizar ausência com -1
### Para colunas com valores negativos, criar flag de missing e imputar com mediana

In [137]:
var_cols = [col for col in df.columns if col.startswith('VAR_')]

global_neg_cols = [col for col in var_cols if (df[col] < 0).any()]
other_cols = [col for col in var_cols if col not in global_neg_cols]
df[other_cols] = df[other_cols].fillna(-1)

for col in global_neg_cols:
    df[f"{col}_missing"] = df[col].isna().astype(int)
    median_val = df[col].median()
    df[col] = df[col].fillna(median_val)

## Análise de completude das features

In [138]:
counts = {}
for col in var_cols:
    counts[col] = {
        0: int((df[col] == 0).sum()),
        -1: int((df[col] == -1).sum())
    }
counts_df = pd.DataFrame.from_dict(counts, orient='index').rename(columns={0: 'count_0', -1: 'count_-1'})
total = len(df)
modes = {col: df[col].mode().iloc[0] if not df[col].mode().empty else np.nan for col in var_cols}
mode_counts = {col: int((df[col] == mode_val).sum()) for col, mode_val in modes.items()}
modes_df = pd.DataFrame({'mode': pd.Series(modes), 'count_mode': pd.Series(mode_counts)})
completeness_df = counts_df.merge(modes_df, left_index=True, right_index=True)
completeness_df['total'] = total
completeness_df['pc_0'] = (completeness_df['count_0'] / total).round(2)
completeness_df['pc_-1'] = (completeness_df['count_-1'] / total).round(2)
completeness_df['pc_moda'] = (completeness_df['count_mode'] / total).round(2)
completeness_df
#completeness_df.to_excel("completude.xlsx")

,count_0,count_-1,mode,count_mode,total,pc_0,pc_-1,pc_moda
VAR_1,3376,416,0.00,3376,10738,0.31,0.04,0.31
VAR_2,9125,416,0.00,9125,10738,0.85,0.04,0.85
VAR_3,8528,416,0.00,8528,10738,0.79,0.04,0.79
VAR_4,7163,416,0.00,7163,10738,0.67,0.04,0.67
VAR_5,0,4060,-1.00,4060,10738,0.00,0.38,0.38
...,...,...,...,...,...,...,...,...
VAR_74,3932,5541,-1.00,5541,10738,0.37,0.52,0.52
VAR_75,0,7024,-1.00,7024,10738,0.00,0.65,0.65
VAR_76,112,0,318.28,4552,10738,0.01,0.00,0.42
VAR_77,2677,5565,-1.00,5565,10738,0.25,0.52,0.52


In [ ]:
# Separação em DEV e OOT

df["safra"] = df["safra"].astype(str)

df["period_label"] = pd.Series(index=df.index, dtype="object")

mask_dev = (df["safra"] >= "201402") & (df["safra"] <= "201409")
mask_oot = (df["safra"] >= "201410") & (df["safra"] <= "201412")

df.loc[mask_dev, "period_label"] = "DEV"
df.loc[mask_oot, "period_label"] = "OOT"

dev = df[df["period_label"] == "DEV"].copy()
oot = df[df["period_label"] == "OOT"].copy()

## PSI E IV

In [ ]:
# Cálculo de PSI
def psi(expected, actual, bins=10):
    breaks = np.unique(np.percentile(expected, np.linspace(0, 100, bins + 1)))
    exp_perc = np.histogram(expected, bins=breaks)[0] / len(expected)
    act_perc = np.histogram(actual,   bins=breaks)[0] / len(actual)
    exp_perc = np.where(exp_perc == 0, 1e-8, exp_perc)
    act_perc = np.where(act_perc == 0, 1e-8, act_perc)
    return np.sum((exp_perc - act_perc) * np.log(exp_perc / act_perc))

psi_results = []
for c in var_cols:
    psi_results.append({
        "variable": c,
        "psi_dev": 0.0,
        "psi_oot": psi(dev[c].values, oot[c].values, bins=10)
    })
psi_df = pd.DataFrame(psi_results).sort_values("psi_oot", ascending=False)
psi_df.to_excel("psi_df.xlsx")
psi_df

,variable,psi_dev,psi_oot
52,VAR_53,0.0,0.859218
29,VAR_30,0.0,0.318724
53,VAR_54,0.0,0.144043
0,VAR_1,0.0,0.099746
16,VAR_17,0.0,0.092494
...,...,...,...
24,VAR_25,0.0,0.000953
76,VAR_77,0.0,0.000942
11,VAR_12,0.0,0.000558
7,VAR_8,0.0,0.000494


In [143]:
excl_psi = ['VAR_53'
'VAR_54',
'VAR_30',
'VAR_1', 'VAR_2', 'VAR_3', 'VAR_4'
]
#'VAR_1', 'VAR_2', 'VAR_3', 'VAR_4', 'VAR_7', 'VAR_9', 'VAR_14', 'VAR_15', 'VAR_17', 'VAR_20', 'VAR_26', 'VAR_27', 'VAR_29', 'VAR_30', 'VAR_31', 'VAR_34', 'VAR_37', 'VAR_38', 'VAR_41', 'VAR_42', 'VAR_46', 'VAR_49', 'VAR_50', 'VAR_51', 'VAR_52', 'VAR_53', 'VAR_54', 'VAR_56', 'VAR_57', 'VAR_60', 'VAR_62', 'VAR_65', 'VAR_67', 'VAR_71', 'VAR_73', 'VAR_74', 'VAR_75', 'VAR_78', 'month_sin', 'month_cos'

In [ ]:
# Cálculo de IV
def calc_iv(series, target, bins=10):
    df2 = pd.DataFrame({"x": series, "y": target})
    df2["bin"] = pd.qcut(df2["x"], q=bins, duplicates="drop")
    grp = df2.groupby("bin")["y"].agg(["count","sum"]).rename(columns={"sum":"bad","count":"total"})
    grp["good"] = grp["total"] - grp["bad"]
    dist_good = grp["good"] / grp["good"].sum()
    dist_bad  = grp["bad"]  / grp["bad"].sum()
    dist_good = dist_good.replace(0, 1e-8)
    dist_bad  = dist_bad.replace(0, 1e-8)
    woe = np.log(dist_good / dist_bad)
    return ((dist_good - dist_bad) * woe).sum()

iv_results = []
for label, subset in [("dev", dev), ("oot", oot)]:
    for c in var_cols:
        iv_results.append({
            "variable": c,
            f"iv_{label}": calc_iv(subset[c], subset["y"], bins=10)
        })
iv_df = (pd.DataFrame(iv_results)
         .pivot_table(index="variable", values=["iv_dev","iv_oot"])
         .reset_index()
         .sort_values("iv_dev", ascending=False))
iv_df

,variable,iv_dev,iv_oot
0,VAR_1,0.823976,0.139079
22,VAR_3,0.656535,0.071447
11,VAR_2,0.634040,0.048174
33,VAR_4,0.556593,0.099429
8,VAR_17,0.198135,0.114150
...,...,...,...
47,VAR_52,0.003939,0.009523
51,VAR_56,0.003491,0.034138
29,VAR_36,0.002297,0.000104
57,VAR_61,0.000075,0.005044


## Feature Selection com Boruta

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

X = df[var_cols].values
y = df['y'].values

# Estimador base
rf = RandomForestClassifier(max_depth=2, random_state=42, n_jobs=-1)

# Configurar Boruta
selector = BorutaPy(
    estimator=rf,
    n_estimators='auto',
    alpha=0.01,
    verbose=2,
    random_state=42
)

selector.fit(X, y)

# Recuperar features selecionadas
selected_features = [var_cols[i] for i, keep in enumerate(selector.support_) if keep]
print("Features selecionadas pelo Boruta:", selected_features)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	78
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	78
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	78
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	78
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	78
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	78
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	78
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	0
Tentative: 	78
Rejected: 	0
Iteration: 	9 / 100
Confirmed: 	0
Tentative: 	78
Rejected: 	0
Iteration: 	10 / 100
Confirmed: 	54
Tentative: 	24
Rejected: 	0
Iteration: 	11 / 100
Confirmed: 	54
Tentative: 	24
Rejected: 	0
Iteration: 	12 / 100
Confirmed: 	54
Tentative: 	22
Rejected: 	2
Iteration: 	13 / 100
Confirmed: 	54
Tentative: 	22
Rejected: 	2
Iteration: 	14 / 100
Confirmed: 	54
Tentative: 	22
Rejected: 	2
Iteration: 	15 / 100
Confirmed: 	55
Tentative: 	21
Rejected: 	2
Iteration: 	16 / 100
Confirmed: 	55
Tentative: 	18
Rejecte

In [146]:
cols_boruta = ['VAR_1', 'VAR_2', 'VAR_3', 'VAR_4', 'VAR_6', 'VAR_9', 'VAR_10', 'VAR_12', 'VAR_14', 'VAR_16', 'VAR_17', 
'VAR_18', 'VAR_19', 'VAR_20', 'VAR_21', 'VAR_22', 'VAR_23', 'VAR_25', 'VAR_26', 'VAR_27', 'VAR_28', 'VAR_29', 'VAR_30', 
'VAR_31', 'VAR_32', 'VAR_33', 'VAR_34', 'VAR_37', 'VAR_38', 'VAR_40', 'VAR_41', 'VAR_42', 'VAR_43', 'VAR_44', 'VAR_46', 
'VAR_47', 'VAR_48', 'VAR_49', 'VAR_51', 'VAR_53', 'VAR_57', 'VAR_59', 'VAR_60', 'VAR_62', 'VAR_64', 'VAR_65', 'VAR_66', 
'VAR_67', 'VAR_68', 'VAR_69', 'VAR_70', 'VAR_71', 'VAR_72', 'VAR_73', 'VAR_74', 'VAR_76', 'VAR_77', 'VAR_78']

cols_boruta = [col for col in cols_boruta if col not in excl_psi]
